In [1]:
# !pip install geopandas
# !pip install geopy
# !pip install tqdm
# !pip install plotly_express

In [2]:
# %load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook

In [20]:
def get_location(lat, long):
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat) + ", " + str(long)
    return locator.reverse(coordinates)

def set_value(state, postcode, county):
    value = {'$set': {'state': state , \
                        'postcode': postcode, \
                          'county': county}}
    return value

In [38]:
# Example
location = get_location("-1.2841","36.8155")
location.raw

{'place_id': 103334635,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 4741916,
 'lat': '-1.2838893632221897',
 'lon': '36.81592321618731',
 'display_name': 'Uhuru Highway, Upper Hill, Nairobi, 46464, Kenya',
 'address': {'road': 'Uhuru Highway',
  'suburb': 'Upper Hill',
  'city': 'Nairobi',
  'state': 'Nairobi',
  'ISO3166-2-lvl4': 'KE-30',
  'region': 'Nairobi',
  'postcode': '46464',
  'country': 'Kenya',
  'country_code': 'ke'},
 'boundingbox': ['-1.2864162', '-1.2819523', '36.8149821', '36.8171431']}

In [10]:
## I usually import data to MongoDB first
## then pull the data from there. 
## you may write your own program to fetch the data. 

import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient['Side_Projects']
tbl = db['Reverse_geocodes_2']
tbl.find().skip(1440).count()

<ipython-input-10-465cacbddb37>:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  tbl.find().skip(1440).count()


1839

In [15]:
docs = tbl.find().skip(1438)
count = 0
for doc in docs:
    lat, long = 0, 0
    lat = doc.get("LocationLatitude")
    long = doc.get("LocationLongitude")
    count += 1
count

401

In [13]:
docs.count()

<ipython-input-13-5dde81ecd439>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  docs.count()


1839

In [3]:
docs = tbl.find().skip(1438)

count = 0
for doc in docs:
    lat, long = -999, 999
    lat = doc.get("LocationLatitude")
    long = doc.get("LocationLongitude")
    
    location = get_location(lat, long)
    state = location.raw.get('address').get('state')
    postcode = location.raw.get('address').get('postcode')
    county = location.raw.get('address').get('county')
    tbl.update_one({'_id': doc['_id']}, set_value(state, postcode, county))
    count += 1
    
    if count % 20 == 0: 
        print('{} - documents inserted'.format(count))
    

In [34]:
import requests
import json
import time

'''
there is a limit you can make request in a certain amount of time
when I ran the program, I get connection time out several times. 
you can catch the error and make the requests again. 
I have to change my ip and make the requests 😏. 

'''
def get_location_by_ip(ip):
    time.sleep(1)
    request_url = 'https://geolocation-db.com/jsonp/' + ip
    response = requests.get(request_url)
    result = response.content.decode()
    result = result.split("(")[1].strip(")")
    result  = json.loads(result)
    return result

def set_value2(state, postcode, city, country_name):
    value = {'$set': {'state': state , \
                        'postcode': postcode, \
                          'city': city, \
                             'country_name': country_name}}
    return value

In [35]:
docs = tbl.find().skip(1829)

count = 0
for doc in docs:
    
    ip = doc.get("IPAddress")
    
    location = get_location_by_ip(ip)
    state = location.get('state')
    postcode = location.get('postal')
    city = location.get('city')
    country_name = location.get('country_name')
    tbl.update_one({'_id': doc['_id']}, set_value2(state, postcode, city, country_name))
    count += 1
    
    if count % 20 == 0: 
        print('{} - documents inserted'.format(count))